**https://github.com/pdrm83/sent2vec**

**https://colab.research.google.com/github/kakshay21/sentence_embeddings/blob/master/word_to_sentence_embedding.ipynb**

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries**

In [ ]:
import os
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools
from itertools import chain, product
from itertools import zip_longest

**Load Datasets**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Train.csv')[0:100]
Test_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Test.csv')[0:100]
Val_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Val.csv')[0:100]

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((100, 3), (100, 3), (100, 2))

**Text Preprocessing**

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
Train_data['cleanText'] = Train_data['tweet'].map(lambda s:preprocess(s))
Val_data['cleanText'] = Val_data['tweet'].map(lambda s:preprocess(s))
Test_data['cleanText'] = Test_data['tweet'].map(lambda s:preprocess(s))

In [ ]:
Train_data.head()

,id,tweet,label,cleanText
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...


In [ ]:
Val_data.head()

,id,tweet,label,cleanText
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...


In [ ]:
Test_data.head()

,id,tweet,cleanText
0,1,Our daily update is published. States reported...,daily update published states reported tests n...
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((100, 4), (100, 4), (100, 3))

**Sent2Vec**

In [ ]:
!pip3 install sent2vec

     |████████████████████████████████| 3.4 MB 7.0 MB/s 
     |████████████████████████████████| 895 kB 43.0 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
     |████████████████████████████████| 3.3 MB 61.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

In [ ]:
def sentence2vec(sentences):
    for i in range(len(sentences)):
        vectorizer = Vectorizer()
        vectorizer.bert(sentences)
        vectors_bert = vectorizer.vectors
        return vectors_bert

In [ ]:
X_train = sentence2vec(Train_data['cleanText'])
X_test = sentence2vec(Val_data['cleanText'])

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(Train_data['label'])
y_test = label_encoder.transform(Val_data['label'])
label_encoder.classes_

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- T

array(['fake', 'real'], dtype=object)

In [ ]:
X_train

array([[-0.16795208, -0.13445276,  0.36343086, ..., -0.02985514,
         0.40898624, -0.00704629],
       [-0.02180064, -0.29588148,  0.41348478, ..., -0.0020552 ,
         0.41477576, -0.08852363],
       [-0.16262645, -0.34111977,  0.22103094, ..., -0.01759622,
         0.33573166,  0.07020195],
       ...,
       [-0.22935632, -0.33637303,  0.28155524, ..., -0.00152714,
         0.24690866, -0.01398035],
       [-0.36097202, -0.25857863,  0.29262745, ...,  0.09183461,
         0.29099384,  0.06561382],
       [-0.23507966, -0.2294102 ,  0.23781638, ...,  0.07497717,
         0.24459267,  0.09490472]], dtype=float32)

In [ ]:
X_test

array([[ 0.0945581 , -0.27793142,  0.21440421, ...,  0.01385815,
         0.721489  , -0.10920011],
       [-0.12370685, -0.32075247,  0.2729642 , ..., -0.03411711,
         0.4814817 , -0.09401319],
       [ 0.03580489, -0.28000844,  0.16588867, ...,  0.01598789,
         0.6949182 ,  0.0202848 ],
       ...,
       [ 0.00764847, -0.3534844 ,  0.3715629 , ...,  0.02818328,
         0.5887341 ,  0.02341761],
       [-0.05338405, -0.28519583,  0.35651523, ...,  0.00951349,
         0.5435827 ,  0.02745817],
       [ 0.1487389 , -0.22907831,  0.2112751 , ...,  0.01804679,
         0.7338797 , -0.06559254]], dtype=float32)

In [ ]:
y_train

array([1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1])

In [ ]:
y_test

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0])

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((100, 768), (100, 768), (100,), (100,))

**Model Evaluation**

In [ ]:
dict_classifiers = {
    "Nearest Neighbors":KNeighborsClassifier(3),
    "LogisticRegression":LogisticRegression(),
    #"NuSVC":NuSVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "MLP": MLPClassifier(alpha=1, max_iter=1000),
    "AdaBoost":AdaBoostClassifier(),
    "GaussianNB":GaussianNB(),
    "LinearDiscriminant":LinearDiscriminantAnalysis(),
    "QuadraticDiscriminant":QuadraticDiscriminantAnalysis(),
    "GradientBoost":GradientBoostingClassifier(),
    #"MultinomialNB":MultinomialNB(),
    "SGD":SGDClassifier(),
    "LGBM":LGBMClassifier(),
    #"XGB":XGBClassifier(),
    "Gaussian Process Classifier":GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Support Vector Machine 1":SVC(kernel="linear", C=0.025),
    "Support Vector Machine 2":SVC(gamma=2, C=1)
}

In [ ]:
def evaluation_matrix(model, testX, testy):
    # predict probabilities for test set
    yhat_probs = model.predict(testX)
    #print(yhat_probs)
    # predict crisp classes for test set
    #yhat_classes = model.predict_classes(testX)
    # reduce to 1d array
    #yhat_probs = yhat_probs[:, 0]
    #print(yhat_probs)
    #yhat_classes = yhat_classes
    yhat_classes = yhat_probs
    #accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(testy, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testy, yhat_classes, average='weighted')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testy, yhat_classes, average='weighted')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testy, yhat_classes, average='weighted')
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(testy, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    #auc = roc_auc_score(testy, yhat_probs, multi_class = 'ovr')
    #print('ROC AUC: %f' % auc)
    #confusion matrix
    matrix = confusion_matrix(testy, yhat_classes)
    print(matrix)
    return accuracy, precision, recall, f1, kappa

In [ ]:
def batch_classify(dict_classifiers, X_train,X_test,y_train,y_test):
    no_classifiers = len(dict_classifiers)
    dict_super_models = {}

    for classifier_name,classifier in list(dict_classifiers.items())[:no_classifiers]:
        print(classifier_name)
        classifier.fit(X_train, y_train)
        train_score = classifier.score(X_train, y_train)
        test_score = classifier.score(X_test, y_test)
        accuracy, precision, recall, f1, kappa = evaluation_matrix(classifier, X_test, y_test)
        y_pred = classifier.predict(X_test)
        print('train score:{%0.2f} test_score:{%0.2f}' %(train_score, test_score))
        dict_super_models[classifier_name] = {'Supervised': classifier_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Kappa': kappa}   

    return dict_super_models

In [ ]:
dict_super_models = batch_classify(dict_classifiers, X_train,X_test,y_train,y_test)

Nearest Neighbors
Accuracy: 0.700000
Precision: 0.735056
Recall: 0.700000
F1 score: 0.699040
Cohens kappa: 0.414748
[[36  7]
 [23 34]]
train score:{0.91} test_score:{0.70}
LogisticRegression
Accuracy: 0.810000
Precision: 0.809500
Recall: 0.810000
F1 score: 0.809002
Cohens kappa: 0.609053
[[32 11]
 [ 8 49]]
train score:{0.96} test_score:{0.81}
Decision Tree
Accuracy: 0.630000
Precision: 0.628990
Recall: 0.630000
F1 score: 0.629432
Cohens kappa: 0.243044
[[24 19]
 [18 39]]
train score:{1.00} test_score:{0.63}
Random Forest
Accuracy: 0.670000
Precision: 0.667750
Recall: 0.670000
F1 score: 0.668267
Cohens kappa: 0.320988
[[25 18]
 [15 42]]
train score:{0.96} test_score:{0.67}
MLP
Accuracy: 0.790000
Precision: 0.789250
Recall: 0.790000
F1 score: 0.788897
Cohens kappa: 0.567901
[[31 12]
 [ 9 48]]
train score:{1.00} test_score:{0.79}
AdaBoost
Accuracy: 0.840000
Precision: 0.840000
Recall: 0.840000
F1 score: 0.840000
Cohens kappa: 0.673603
[[35  8]
 [ 8 49]]
train score:{1.00} test_score:{0.84

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Accuracy: 0.700000
Precision: 0.701683
Recall: 0.700000
F1 score: 0.689961
Cohens kappa: 0.366286
[[22 21]
 [ 9 48]]
train score:{1.00} test_score:{0.70}
SGD
Accuracy: 0.700000
Precision: 0.803448
Recall: 0.700000
F1 score: 0.650893
Cohens kappa: 0.330656
[[13 30]
 [ 0 57]]
train score:{0.82} test_score:{0.70}
LGBM
Accuracy: 0.750000
Precision: 0.750812
Recall: 0.750000
F1 score: 0.750331
Cohens kappa: 0.491456
[[31 12]
 [13 44]]
train score:{1.00} test_score:{0.75}
Gaussian Process Classifier
Accuracy: 0.790000
Precision: 0.789250
Recall: 0.790000
F1 score: 0.788897
Cohens kappa: 0.567901
[[31 12]
 [ 9 48]]
train score:{1.00} test_score:{0.79}
Support Vector Machine 1
Accuracy: 0.780000
Precision: 0.813600
Recall: 0.780000
F1 score: 0.765882
Cohens kappa: 0.526882
[[23 20]
 [ 2 55]]
train score:{0.83} test_score:{0.78}
Support Vector Machine 2
Accuracy: 0.570000
Precision: 0.324900
Recall: 0.570000
F1 score: 0.413885
Cohens kappa: 0.000000
[[ 0 43]
 [ 0 57]]
train score:{1.00} test_sc

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def display_dict_models(dict_models, sort_by='Accuracy'):
#def display_dict_models(dict_models, sort_by='train_score'):
    cls = [key for key in dict_models.keys()]
    accuracy = [dict_models[key]['Accuracy'] for key in cls]
    precision = [dict_models[key]['Precision'] for key in cls]
    recall = [dict_models[key]['Recall'] for key in cls]
    f1 = [dict_models[key]['F1'] for key in cls]
    kappa = [dict_models[key]['Kappa'] for key in cls]
    #training_t = [dict_models[key]['train_time'] for key in cls]
    
    #df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),6)), columns = ['classifier', 'Accuracy', 'Precision', 'Recall', 'F1', 'Kappa'])
    # df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),3)), columns = ['classifier', 'train_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'Accuracy'] = accuracy[ii]
        df_.loc[ii, 'Precision'] = precision[ii]
        df_.loc[ii, 'Recall'] = recall[ii]
        df_.loc[ii, 'F1'] = f1[ii]
        df_.loc[ii, 'Kappa'] = kappa[ii]
       
        #df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    return df_

In [ ]:
df = display_dict_models(dict_super_models)

,classifier,Accuracy,Precision,Recall,F1,Kappa
5,AdaBoost,0.84,0.840000,0.84,0.840000,0.673603
1,LogisticRegression,0.81,0.809500,0.81,0.809002,0.609053
4,MLP,0.79,0.789250,0.79,0.788897,0.567901
12,Gaussian Process Classifier,0.79,0.789250,0.79,0.788897,0.567901
13,Support Vector Machine 1,0.78,0.813600,0.78,0.765882,0.526882
11,LGBM,0.75,0.750812,0.75,0.750331,0.491456
7,LinearDiscriminant,0.74,0.762788,0.74,0.740624,0.487179
0,Nearest Neighbors,0.70,0.735056,0.70,0.699040,0.414748
9,GradientBoost,0.70,0.701683,0.70,0.689961,0.366286
10,SGD,0.70,0.803448,0.70,0.650893,0.330656


In [ ]:
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                  classifier &  Accuracy &  Precision &  Recall &        F1 &     Kappa \\
\midrule
           Nearest Neighbors &      0.70 &   0.735056 &    0.70 &  0.699040 &  0.414748 \\
          LogisticRegression &      0.81 &   0.809500 &    0.81 &  0.809002 &  0.609053 \\
               Decision Tree &      0.63 &   0.628990 &    0.63 &  0.629432 &  0.243044 \\
               Random Forest &      0.67 &   0.667750 &    0.67 &  0.668267 &  0.320988 \\
                         MLP &      0.79 &   0.789250 &    0.79 &  0.788897 &  0.567901 \\
                    AdaBoost &      0.84 &   0.840000 &    0.84 &  0.840000 &  0.673603 \\
                  GaussianNB &      0.64 &   0.703826 &    0.64 &  0.631273 &  0.312715 \\
          LinearDiscriminant &      0.74 &   0.762788 &    0.74 &  0.740624 &  0.487179 \\
       QuadraticDiscriminant &      0.60 &   0.595965 &    0.60 &  0.597024 &  0.174577 \\
               GradientBoost &      0.70 &   0.7